Web scraping conference call transcripts from Seeking Alpha <br>

For a searched-for stock:
    https://seekingalpha.com/search?q=atra&type=keyword&tab=transcripts # 10 links per page. Not all are transcripts
    <a class="item-ticker" href="/symbol/ATRA"><strong>ATRA</strong></a> # for each link, this can be used to identify the article as focused on the stock in question (rather than the stock referenced in another stock's report). 
    <a href="/author/sa-transcripts">SA Transcripts</a> This can be used to ensure that the link is for a transcript (rather than a news article).

Each stock can be has a transcript page: https://seekingalpha.com/symbol/RARE/transcripts <br>
However, these also include slideshows and presentation decks. <br>
All earnings call transcripts are distinguished by the phrase "Earnings Call Transcript" within <a class="__29a76-3a6i8" href="/article/4424388-ultragenyx-pharmaceutical-inc-s-rare-ceo-emil-kakkis-on-q1-2021-results-earnings-call?source=content_type%3Areact%7Csection%3ATranscripts%7Csection_asset%3ATranscripts%7Cfirst_level_url%3Asymbol%7Cbutton%3ATitle%7Clock_status%3ANo%7Cline%3A1">Ultragenyx Pharmaceutical Inc.'s (RARE) CEO Emil Kakkis on Q1 2021 Results - Earnings Call Transcript</a>

As it stands, my strategy will be to:
* Use yfinance API to identify 1,000 stocks that (a) have been incorporated for 5 years and (b) are in the same sector. 
* Collect tickers from these stocks
* Loop over the ticker labels, years, and quarters to scrape transcript data from Seeking Alpha
* 

In [7]:
import requests
import time
from numpy import random
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from datetime import datetime
from datetime import date
import operator

In [48]:
chrome_options = Options()
#chrome_options.add_argument("--incognito")
chrome_options.add_argument("--enable-javascript")
driver = webdriver.Chrome(options=chrome_options, executable_path='/opt/anaconda3/envs/metis/lib/python3.8/site-packages/chromedriver_binary/chromedriver')
url = 'https://seekingalpha.com/account/login'
driver.get(url)
time.sleep(2)
driver.add_cookie({"name": "key", "value": "value"})

# Entering my credentials
elements = driver.find_elements_by_xpath('//*[@id="root"]/div[1]/main/div[2]/form/div/div[1]/label/input')
username = elements[0]
username.clear()
user_name = "ptgoff1.618@gmail.com"
username.send_keys(user_name)

elements = driver.find_elements_by_xpath('//*[@id="signInPasswordField"]')
password = elements[0]
password.clear()
password.send_keys("0sYO3@Ju4NiV")

elements = driver.find_elements_by_xpath('//*[@id="root"]/div[1]/main/div[2]/form/button')
submit = elements[0]
submit.click()

In [49]:
energy_tickers = ["XOM" ,"CVX" ,"RDS-A" ,"RDS-B" ,"PTR" ,"TOT" ,"BP" ,"ENB" ,"SNP" ,"COP" ,"EQNR" ,"PBR-A" ,"PBR" ,"EPD" ,"TRP" ,
                  "EOG" ,"E" ,"SLB" ,"KMI" ,"CNQ" ,"MPC" ,"PSX" ,"PXD" ,"SU" ,"VLO" ,"WMB" ,"CSAN" ,"MPLX" ,"HES" ,"ET" ,"OXY" ,
                  "EC" ,"BKR" ,"OKE" ,"HAL" ,"PBA" ,"DVN" ,"CVE" ,"TS" ,"FANG" ,"TPL" ,"CLR" ,"MMP" ,"SSL" ,"MRO" ,"WES" ,"TRGP" ,
                  "APA" ,"PSXP" ,"CCJ" ,"XEC" ,"PAA" ,"COG" ,"OVV" ,"NOV" ,"SHLX" ,"VVV" ,"HFC" ,"SHI" ,"DCP" ,"EQT" ,"CHX" ,"NFG" ,
                  "CHK" ,"AM" ,"UGP" ,"PDCE" ,"FTI" ,"SUN" ,"ETRN" ,"ENBL" ,"MTDR" ,"SWN" ,"HP" ,"MGY" ,"CNX" ,"VNOM" ,"RRC" ,"DEN" ,
                  "MUR" ,"AR" ,"NS-PB" ,"WHD" ,"ENLC" ,"LBRT" ,"REGI" ,"CPG" ,"CVI" ,"RIG" ,"CEQP" ,"BSM" ,"HEP" ,"INT" ,"NS" ,"CRC" ,
                  "SM" ,"CLNE" ,"PBF" ,"DK" ,"PAGP" ,"DKL" ,"EURN" ,"RTLR" ,"CPE" ,"OAS" ,"WLL" ,"FRO" ,"PTEN" ,"YPF" ,"AROC" ,"USAC" ,
                  "BPMP" ,"CLB" ,"OII" ,"GLOG-PA" ,"NBLX" ,"CRK" ,"TGP" ,"ERF" ,"DNOW" ,"DRQ" ,"VET" ,"GLNG" ,"RES" ,"KOS" ,"GEL" ,
                  "XOG" ,"PUMP" ,"STNG" ,"NGL-PB" ,"NGL-PC" ,"VEI" ,"TALO" ,"DHT" ,"MNRL" ,"PBFX" ,"GPRK" ,"GLOP-PC" ,"GLOP-PB" ,"MRC" ,
                  "GLP" ,"NEX" ,"VTOL" ,"FI" ,"SLCA" ,"BOOM" ,"PARR" ,"BCEI" ,"OMP" ,"ARCH" ,"CAPL" ,"ARLP" ,"HLX" ,"KRP" ,"ESTE" ,"LPG" ,
                  "NBR" ,"TRMD" ,"TGS" ,"FLNG" ,"WTTR" ,"NVGS" ,"PVAC" ,"TDW" ,"GLOG" ,"HMLP" ,"SRLP" ,"NBR-PA" ,"DMLP" ,"SBR" ,"CLMT" ,
                  "REX" ,"BRY" ,"WTI" ,"TNK" ,"SOI" ,"BTU" ,"DSSI" ,"SGU" ,"LPI" ,"CEIX" ,"TTI" ,"NOA" ,"HESM" ,"OIS" ,"PDS" ,"TK" ,
                  "TNP-PD" ,"TNP-PE" ,"NR" ,"TNP-PF" ,"NGL" ,"BORR" ,"AMR" ,"SJT" ,"NRP" ,"VIST" ,"OSG" ,"RNET" ,"TNP" ,"PBT" ,"NC" ,"TUSK" ,
                  "GLOP" ,"SD" ,"EGY" ,"SLNG" ,"EXTN" ,"SBOW" ,"NGS" ,"FTK" ,"AMPY" ,"GEOS" ,"FET" ,"SND" ,"DLNG" ,"KLXE" ,"RNGR" ,"SMLP" ,
                  "MMLP" ,"DLNG-PB" ,"BPT" ,"CCLP" ,"PRT" ,"PHX" ,"GIFI" ,"MVO" ,"NINE" ,"VOC" ,"CRT" ,"NRT" ,"DWSN" ,"NNA" ,"PVL" ,"IO" ,
                  "CELP" ,"ICD" ,"MARPS" ,"MTR" ,"DCP-PB" ,"TGP-PA" ,"NS-PC" ,"ALIN-PA" ,"ALIN-PE" ,"EP-PC" ,"GLOP-PA" ,"CEQP-P" ,"HMLP-PA" ,
                  "ALIN-PB" ,"DCP-PC" ,"NS-PA" ,"VAL" ,"TGP-PB" ,"DLNG-PA" ,"GLP-PB" ,"GLP-PA" ,"GMLPP" ,"UROY"]

for ticker in energy_tickers:
    # For each ticker, navigate to the page that holds Seeking Alpha's links to transcripts for that include that ticker
    page = 'https://seekingalpha.com/search?q=' + ticker + '&type=keyword&tab=transcripts'
    driver.get(page)
    # Pause for a moment, then send the page code off to beautifulsoup
    time.sleep(random.randint(5,20)/10)
    page_source = driver.page_source
    soup = BeautifulSoup(page_source, 'lxml')

    # For each ticker, we want to collect a list of links to the transcipt data for each quarter, the quarter in question,
    # and corresponding date. We initialize those lists here.
    links = []
    quarters = []
    dates = []
    # Collecting all the links from the first transcript page for each ticker:
    for each in soup.find_all("div",{'class':'item-details'}):
        if (operator.contains(each.find('a').text, "Earnings Call Transcript")) & (operator.contains(each.a.text, ticker)):
            links.append("https://seekingalpha.com" + each.find('a').get("href"))
            calldate = each.find('div',{'class':'item-metadata'}).text.replace('\n', "").replace(ticker, "").replace('SA Transcripts', "")
            for day in ['Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday']:
                if (operator.contains(calldate, day)):
                    n = calldate.find(day)
                    calldate = calldate[n:].strip()
            dates.append(calldate)
            if operator.contains(each.find('a').text.lower(), "q1"):
                quarters.append('q1')
            elif operator.contains(each.find('a').text.lower(), "q2"):
                quarters.append('q2')
            elif operator.contains(each.find('a').text.lower(), "q3"):
                quarters.append('q3')
            else:
                quarters.append('q4')
    
    # Each page contains 10 links. We'd like to have 10 transcript links per ticker. 
    # Some transcripts reference other tickers, so not all 10 links on the initial page will be to transcripts of the 
    # ticker in question. In those scenarios we can dig into the next 10 transcript-link pages. 
    # If we can't get 10 links in the first 10 pages, we'll move on to the next ticker. 
    pagenum = 2
    trans_num = len(links) 
    while (trans_num < 10) & (pagenum < 10):
        page = 'https://seekingalpha.com/search?q=' + ticker + '&type=keyword&tab=transcripts' + '#page=' + str(pagenum)
        driver.get(page)
        time.sleep(random.randint(5,20)/10)
        page_source = driver.page_source
        soup = BeautifulSoup(page_source, 'lxml')

        # Collecting all the links from the subsequent transcript pages for each ticker:
        for each in soup.find_all("div",{'class':'item-details'}):
            if (operator.contains(each.find('a').text, "Earnings Call Transcript")) & (operator.contains(each.a.text, ticker)):
                links.append("https://seekingalpha.com" + each.find('a').get("href"))
                calldate = each.find('div',{'class':'item-metadata'}).text.replace('\n', "").replace(ticker, "").replace('SA Transcripts', "")
                for day in ['Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday']:
                    if (operator.contains(calldate, day)):
                        n = calldate.find(day)
                        calldate = calldate[n:].strip()
                dates.append(calldate)
                if operator.contains(each.find('a').text.lower(), "q1"):
                    quarters.append('q1')
                elif operator.contains(each.find('a').text.lower(), "q2"):
                    quarters.append('q2')
                elif operator.contains(each.find('a').text.lower(), "q3"):
                    quarters.append('q3')
                else:
                    quarters.append('q4')
        pagenum += 1 
        trans_num = len(links)
    
    # A pretty hacky solution to get the ordinal dates converted to datetime:
    blank = ''
    dates = [blank.join(calldate.rsplit('th', 1)) for calldate in dates]
    dates = [blank.join(calldate.rsplit('st', 1)) for calldate in dates]
    dates = [blank.join(calldate.rsplit('nd', 1)) for calldate in dates]
    dates = [blank.join(calldate.rsplit('rd', 1)) for calldate in dates]
    dates = [calldate.replace('Augu ', 'August ') for calldate in dates] 
    dates = [calldate.replace('Satuay', 'Saturday') for calldate in dates]
    dates = [calldate.replace('Moay', 'Monday') for calldate in dates]
    dates = [calldate.replace('Suay', 'Sunday') for calldate in dates] 
    dates = [datetime.strptime(calldate, '%A, %B %d %Y') for calldate in dates]
    dates2 = [datetime.strftime(calldate, '%Y-%m-%d') for calldate in dates]
    
    i=0
    # Now that we have the links to (ideally) 10 call transcripts for the ticker,
    # we can follow those lnks and pull down the transcripts.
    for ind, name in enumerate(links):
        page = links[ind]
        driver.get(page)
        time.sleep(0.5)
        page_source = driver.page_source
        trans_soup = BeautifulSoup(page_source, 'html.parser')
        
        # Here's the transcript: 
        trans = trans_soup.find("div",{'data-test-id':'content-container'})
        
        # We'll hard-code the quarter and date information into the filename:
        filename = ticker + "_" + quarters[ind] + "_" +  dates2[ind][0:4] + dates2[ind][5:7] + dates2[ind][8:]

        # Create one text file for each transcript:
        file = open(r"./energy_transcripts/" + filename.lower() + ".txt", 'w')
        file.write(trans.text)
        file.close
        i+=1
    
    print(ticker, i, 'transcripts recieved')

NS-PC 0 transcripts recieved
ALIN-PA 0 transcripts recieved
ALIN-PE 0 transcripts recieved
EP-PC 0 transcripts recieved
GLOP-PA 0 transcripts recieved
CEQP-P 0 transcripts recieved
HMLP-PA 0 transcripts recieved
ALIN-PB 0 transcripts recieved
DCP-PC 0 transcripts recieved
NS-PA 0 transcripts recieved
VAL 12 transcripts recieved
TGP-PB 0 transcripts recieved
DLNG-PA 0 transcripts recieved
GLP-PB 0 transcripts recieved
GLP-PA 0 transcripts recieved
GMLPP 0 transcripts recieved
UROY 0 transcripts recieved


In [31]:
# Tickers collected by sector using Yahoo Finanace (https://finance.yahoo.com/screener/new)
health_tickers = ["JNJ" ,"UNH" ,"PFE" ,"ABT" ,"ABBV" ,"NVS" ,"MRK" ,"TMO" ,"LLY" ,"DHR" ,
                  "MDT" ,"NVO" ,"DHR-PA" ,"AMGN" ,"BMY" ,"AZN" ,"SNY" ,"CVS" ,"ISRG" ,"ANTM" ,
                  "SYK" ,"GSK" ,"CI" ,"GILD" ,"ZTS" ,"HCA" ,"BDX" ,"MRNA" ,"BSX" ,"HUM" ,
                  "EW" ,"VRTX" ,"ILMN" ,"REGN" ,"TAK" ,"PHG" ,"WBA" ,"ALGN" ,"IDXX" ,"BAX" ,
                  "IQV" ,"BIIB" ,"A" ,"BNTX" ,"VEEV" ,"CNC" ,"ALXN" ,"ZBH" ,"DXCM" ,"ALC" ,
                  "MTD" ,"MCK" ,"BGNE" ,"RMD" ,"LH" ,"RPRX" ,"SGEN" ,"GMAB" ,"WST" ,"ABC" ,
                  "CERN" ,"FMS" ,"TDOC" ,"COO" ,"HZNP" ,"NVCR" ,"WAT" ,"TFX" ,"SNN" ,"BIO-B" ,
                  "DGX" ,"INCY" ,"CTLT" ,"BIO" ,"STE" ,"GRFS" ,"VTRS" ,"HOLX" ,"CRL" ,"PODD" ,
                  "CAH" ,"PPD" ,"TECH" ,"MOH" ,"TXG" ,"ALNY" ,"PKI" ,"XRAY" ,"ELAN" ,"BMRN" ,
                  "GDRX" ,"OSH" ,"DVA" ,"GH" ,"ABMD" ,"ARGX" ,"UHS" ,"MASI" ,"ICLR" ,"RDY" ,
                  "NVAX" ,"TEVA" ,"HSIC" ,"PRAH" ,"QGEN" ,"BRKR" ,"RGEN" ,"PEN" ,"BHC" ,"JAZZ" ,
                  "MRVI" ,"CGC" ,"UTHR" ,"EHC" ,"SYNH" ,"NBIX" ,"AMED" ,"NTRA" ,"RARE" ,"ABCL" ,
                  "CHE" ,"MRTX" ,"BBIO" ,"EXEL" ,"HRC" ,"GMED" ,"NVST" ,"ASND" ,"CHNG" ,"THC" ,
                  "ARWR" ,"SHC" ,"DNLI" ,"HALO" ,"RCM" ,"LHCG" ,"IART" ,"IBRX" ,"OMCL" ,"SRPT" ,
                  "MEDP" ,"PRGO" ,"ACHC" ,"SGFY" ,"HQY" ,"NVTA" ,"ONEM" ,"NVRO" ,"VIR" ,"SEM" ,
                  "IONS" ,"ALLK" ,"BPMC" ,"INSP" ,"TWST" ,"NEOG" ,"ADPT" ,"SWAV" ,"GLPG" ,"ALHC" ,
                  "APHA" ,"INOV" ,"MPLN" ,"OCDX" ,"PACB" ,"NARI" ,"SDGR" ,"ABCM" ,"ENSG" ,"OSCR" ,
                  "QDEL" ,"PGNY" ,"CERT" ,"ICUI" ,"PINC" ,"BHVN" ,"AMN" ,"BEAM" ,"ALLO" ,"CVET" ,
                  "CNMD" ,"LIVN" ,"HCM" ,"INNV" ,"AGIO" ,"CMD" ,"GKOS" ,"APLS" ,"ARNA" ,"MMSI" ,
                  "LEGN" ,"NUVA" ,"PDCO" ,"ALKS" ,"SWTX" ,"ACAD" ,"CLOV" ,"NKTR" ,"INSM" ,"EBS" ,
                  "AMWL" ,"SDC" ,"OPCH" ,"HAE" ,"TPTX" ,"SANA" ,"ITGR" ,"BLI" ,"SGRY" ,"ARVN" ,
                  "MOR" ,"INMD" ,"TARO" ,"PTCT" ,"PCRX" ,"OPK" ,"ITCI" ,"BFLY" ,"TIL" ,"MDRX" ,
                  "OMI" ,"SEER" ,"MGLN" ,"GBT" ,"PBH" ,"ALXO" ,"MD" ,"ACCD" ,"BCRX" ,"TLRY" ,
                  "AXNX" ,"HCSG" ,"HCAT" ,"OM" ,"IRTC" ,"EDIT" ,"PHR" ,"RVMD" ,"AVNS" ,"RUBY" ,
                  "MYGN" ,"MCRB" ,"DRNA" ,"MODV" ,"DCPH" ,"NGM" ,"BLUE" ,"MDGL" ,"RCUS" ,"CCXI" ,
                  "PRLD" ,"FGEN" ,"IGMS" ,"AVIR" ,"MGNX" ,"GBIO" ,"MYOV" ,"SILK" ,"ADCT" ,"LMNX" ,
                  "IRWD" ,"IMCR" ,"CYH" ,"EVH" ,"ACB" ,"CYTK" ,"KURA" ,"REPL" ,"EAR" ,"IMVT" ,"ARQT" ,
                  "PMVP" ,"CSII" ,"AMTI" ,"DSGN" ,"USPH" ,"LUNG" ,"ADUS" ,"SGMO" ,"RGNX" ,"QURE" ,"NFH" ,
                  "INGN" ,"EWTX" ,"CCCC" ,"KRON" ,"TBIO" ,"ALVR" ,"IMGN" ,"PGEN" ,"ZYME" ,"ALEC" ,"STTK" ,
                  "INO" ,"ZEAL" ,"CDXS" ,"PETQ" ,"CGEM" ,"ENDP" ,"CRY" ,"CMPS" ,"NXGN" ,"NRC" ,"BKD" ,
                  "ACRS" ,"ATEC" ,"TVTY" ,"RPTX" ,"INVA" ,"YMAB" ,"STOK" ,"PLRX" ,"ATRI" ,"PNTG" ,"CRTX" ,
                  "SNDL" ,"ATRA" ,"PRAX" ,"ALGS" ,"KNTE" ,"ENTA" ,"MRSN" ,"CNST" ,"SRRK" ,"KNSA" ,"HNGR" ,
                  "RAD" ,"APR" ,"VREX" ,"PHAT" ,"PRTA" ,"BDTX" ,"AKRO" ,"ANGO" ,"FDMT" ,"RXDX" ,"PCVX" ,
                  "VOR" ,"PASG" ,"NTUS" ,"AMPH" ,"AMRX" ,"ORIC" ,"OLMA" ,"OFIX" ,"NKTX" ,"HEXO" ,"HSTM" ,
                  "VIVO" ,"TSHA" ,"DYN" ,"BOLT" ,"PHVS" ,"BVS" ,"MESO" ,"TCRR" ,"GRCL" ,"COLL" ,"ADAP" ,
                  "ANNX" ,"SRDX" ,"GTHX" ,"RFL" ,"VYNE" ,"KDNY" ,"SNDX" ,"HARP" ,"ANAB" ,"OSUR" ,
                  "EPZM" ,"CALT" ,"BCYC" ,"ATHA" ,"DBVT" ,"ACHL" ,"ZIOP" ,"ORTX" ,"GOSS" ,"LXRX" ,
                  "ISEE" ,"KDMN" ,"CO" ,"PETS" ,"FNCH" ,"IDYA" ,"SPNE" ,"RLMD" ,"KPTI" ,"GTS" ,"CLVS" ,
                  "MGTX" ,"TARS" ,"ANIK" ,"TLMD" ,"OGI" ,"ICPT" ,"RIGL" ,"CRNX" ,"CUTR" ,"PSTX" ,"CTMX" ,
                  "XBIT" ,"NBTX" ,"EVLO" ,"VAPO" ,"DTIL" ,"OYST" ,"HOOK" ,"AKUS" ,"ANGN" ,"CPSI" ,"PRVB" ,
                  "SPPI" ,"MTEM" ,"AMYT" ,"ARAY" ,"GERN" ,"SGTX" ,"ATNX" ,"GRTS" ,"VIRX" ,"SLDB" ,"SIEN" ,
                  "TERN" ,"PBYI" ,"INZY" ,"OPT" ,"TXMD" ,"FUSN" ,"AVRO" ,"SPRB" ,"BCEL" ,"SPRO" ,"FREQ" ,
                  "FLDM" ,"AFIB" ,"IPHA" ,"BDSI" ,"APYX" ,"UTMD" ,"JNCE" ,"ORPH" ,"SYRS" ,"FRLN" ,"KLDO" ,
                  "LVTX" ,"KALA" ,"BLU" ,"SQZ" ,"CSLT" ,"FIXX" ,"KMDA" ,"CBAY" ,"NH" ,"IVC" ,"AUTL" ,"COGT" ,
                  "KZR" ,"IMUX" ,"UBX" ,"ABUS" ,"CHMA" ,"CABA" ,"OVID" ,"NXTC" ,"NUVB" ,"TCDA" ,"LHDX" ,"INFI" ,
                  "OSMT" ,"LCI" ,"SRGA" ,"NCNA" ,"DBTX" ,"GNFT" ,"OBSV" ,"XERS" ,"MIST" ,"VYGR" ,"OPTN" ,
                  "SBBP" ,"SCPH" ,"ERYP" ,"CCM" ,"CALA" ,"SIOX" ,"GLTO" ,"ENZ" ,"ASMB" ,"IMRA" ,"ODT" ,"OTIC" ,
                  "IFRX" ,"CSU" ,"APRE" ,"SVRA" ,"APTX" ,"CYCN" ,"ARAV" ,"ECOR" ,"GEG" ,"TRIB" ,"ALNA" ,
                  "ACOR" ,"CPIX" ,"NBRV" ,"TLGT" ,"PTIX" ,"AGTI" ,"HOWL" ,"AVAH" ,"VACC" ,"BDXB" ,"AKYA" ,
                  "CHNGU" ,"PALI" ,"AGL" ,"DHR-PB" ,"BSX-PA" ,"BMEA" ,"RAIN" ,"TMCI" ,"RXRX" ,"ELAT" ,"NUWE" ,"VECT" ,"PRVA"]

for ticker in health_tickers:
    # For each ticker, navigate to the page that holds Seeking Alpha's links for transcripts that include that ticker
    page = 'https://seekingalpha.com/search?q=' + ticker + '&type=keyword&tab=transcripts'
    driver.get(page)
    # Pause for a moment, then send the page code off to beautifulsoup
    time.sleep(random.randint(5,20)/10)
    page_source = driver.page_source
    soup = BeautifulSoup(page_source, 'lxml')

    # For each ticker, we want to collect a list of links to the transcipt data for each quarter, the quarter in question,
    # and corresponding date. We initialize those lists here.
    links = []
    quarters = []
    dates = []
    # Collecting all the links from the first transcript page for each ticker:
    for each in soup.find_all("div",{'class':'item-details'}):
        if (operator.contains(each.find('a').text, "Earnings Call Transcript")) & (operator.contains(each.a.text, ticker)):
            links.append("https://seekingalpha.com" + each.find('a').get("href"))
            calldate = each.find('div',{'class':'item-metadata'}).text.replace('\n', "").replace(ticker, "").replace('SA Transcripts', "")
            calldate = calldate.replace('S Transcripts', "")
            for day in ['Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday']:
                if (operator.contains(calldate, day)):
                    n = calldate.find(day)
                    calldate = calldate[n:].strip()
            dates.append(calldate)
            if operator.contains(each.find('a').text.lower(), "q1"):
                quarters.append('q1')
            elif operator.contains(each.find('a').text.lower(), "q2"):
                quarters.append('q2')
            elif operator.contains(each.find('a').text.lower(), "q3"):
                quarters.append('q3')
            else:
                quarters.append('q4')
    
    # Each page contains 10 links. We'd like to have 10 transcript links per ticker. 
    # Some transcripts reference other tickers, so not all 10 links on the initial page will be to transcripts of the 
    # ticker in question. In those scenarios we can dig into the next 10 transcript-link pages. 
    # If we can't get 10 links in the first 10 pages, we'll move on to the next ticker. 
    pagenum = 2
    trans_num = len(links) 
    while (trans_num < 10) & (pagenum < 10):
        page = 'https://seekingalpha.com/search?q=' + ticker + '&type=keyword&tab=transcripts' + '#page=' + str(pagenum)
        driver.get(page)
        time.sleep(random.randint(5,20)/10)
        page_source = driver.page_source
        soup = BeautifulSoup(page_source, 'lxml')

        # Collecting all the links from the subsequent transcript pages for each ticker:
        for each in soup.find_all("div",{'class':'item-details'}):
            if (operator.contains(each.find('a').text, "Earnings Call Transcript")) & (operator.contains(each.a.text, ticker)):
                links.append("https://seekingalpha.com" + each.find('a').get("href"))
                calldate = each.find('div',{'class':'item-metadata'}).text.replace('\n', "").replace(ticker, "").replace('SA Transcripts', "")
                calldate = calldate.replace('S Transcripts', "")
                for day in ['Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday']:
                    if (operator.contains(calldate, day)):
                        n = calldate.find(day)
                        calldate = calldate[n:].strip()
                dates.append(calldate)
                if operator.contains(each.find('a').text.lower(), "q1"):
                    quarters.append('q1')
                elif operator.contains(each.find('a').text.lower(), "q2"):
                    quarters.append('q2')
                elif operator.contains(each.find('a').text.lower(), "q3"):
                    quarters.append('q3')
                else:
                    quarters.append('q4')
        pagenum += 1 
        trans_num = len(links)
    
    # A pretty hacky solution to get the ordinal dates converted to datetime:
    blank = ''
    dates = [blank.join(calldate.rsplit('th', 1)) for calldate in dates]
    dates = [blank.join(calldate.rsplit('st', 1)) for calldate in dates]
    dates = [blank.join(calldate.rsplit('nd', 1)) for calldate in dates]
    dates = [blank.join(calldate.rsplit('rd', 1)) for calldate in dates]
    dates = [calldate.replace('Augu ', 'August ') for calldate in dates] 
    dates = [calldate.replace('Satuay', 'Saturday') for calldate in dates]
    dates = [calldate.replace('Moay', 'Monday') for calldate in dates]
    dates = [calldate.replace('Suay', 'Sunday') for calldate in dates] 
    dates = [datetime.strptime(calldate, '%A, %B %d %Y') for calldate in dates]
    dates2 = [datetime.strftime(calldate, '%Y-%m-%d') for calldate in dates]
    
    i=0
    # Now that we have the links to (ideally) 10 call transcripts for the ticker,
    # we can follow those lnks and pull down the transcripts.
    for ind, name in enumerate(links):
        page = links[ind]
        driver.get(page)
        time.sleep(0.5)
        page_source = driver.page_source
        trans_soup = BeautifulSoup(page_source, 'html.parser')
        
        # Here's the transcript: 
        trans = trans_soup.find("div",{'data-test-id':'content-container'})
        
        # We'll hard-code the quarter and date information into the filename:
        filename = ticker + "_" + quarters[ind] + "_" +  dates2[ind][0:4] + dates2[ind][5:7] + dates2[ind][8:]

        # Create one text file for each transcript:
        file = open(r"./health_transcripts/" + filename.lower() + ".txt", 'w')
        file.write(trans.text)
        file.close
        i+=1
    
    print(ticker, i, 'transcripts recieved')

ACOR 10 transcripts recieved
CPIX 10 transcripts recieved
NBRV 8 transcripts recieved
TLGT 10 transcripts recieved
PTIX 0 transcripts recieved
AGTI 0 transcripts recieved
HOWL 0 transcripts recieved
AVAH 0 transcripts recieved
VACC 0 transcripts recieved
BDXB 0 transcripts recieved
AKYA 0 transcripts recieved
CHNGU 0 transcripts recieved
PALI 0 transcripts recieved
AGL 17 transcripts recieved
DHR-PB 0 transcripts recieved
BSX-PA 0 transcripts recieved
BMEA 0 transcripts recieved
RAIN 0 transcripts recieved
TMCI 0 transcripts recieved
RXRX 0 transcripts recieved
ELAT 0 transcripts recieved
NUWE 0 transcripts recieved
VECT 0 transcripts recieved
PRVA 0 transcripts recieved


In [573]:
tech_tickers = ["AAPL" ,"MSFT" ,"TSM" ,"NVDA" ,"ASML" ,"ORCL" ,"INTC" ,"ADBE" ,"CSCO" ,"CRM" ,"ACN" ,"AVGO" ,"TXN" ,"SAP" ,"QCOM" ,"SHOP" ,"IBM" ,"SONY" ,"AMAT" ,
                "INTU" ,"SQ" ,"NOW" ,"MU" ,"AMD" ,"FIS" ,"LRCX" ,"UBER" ,"FISV" ,"INFY" ,"DELL" ,"VMW" ,"ADSK" ,"ADI" ,"WDAY" ,"TEAM" ,"NXPI" ,"KLAC" ,
                "ERIC" ,"HPQ" ,"TEL" ,"CRWD" ,"CTSH" ,"MCHP" ,"PLTR" ,"WIT" ,"DOCU" ,"SNPS" ,"CDNS" ,"GLW" ,"FTNT" ,"STM" ,"PANW" ,"MSI" ,"MRVL" ,"OKTA" ,"XLNX" ,
                "SWKS" ,"ANSS" ,"NOK" ,"GRMN" ,"KEYS" ,"FTV-PA" ,"EPAM" ,"MXIM" ,"VRSN" ,"ZBRA" ,"ANET" ,"CDW" ,"FTV" ,"CAJ" ,"WORK" ,"FLT" ,"RNG" ,"GIB" ,
                "HUBS" ,"ZS" ,"DDOG" ,"WDC" ,"STX" ,"HPE" ,"TER" ,"QRVO" ,"UMC" ,"APH" ,"BR" ,"TRMB" ,"STNE" ,"SPLK" ,"PAYC" ,"LOGI" ,"SSNC" ,
                "AKAM" ,"NTAP" ,"COUP" ,"CHKP" ,"CLVT" ,"UI" ,"TYL" ,"LYFT" ,"ZEN" ,"TDY" ,"WIX" ,"ON" ,"PTC" ,"NUAN" ,"MPWR" ,"ENTG" ,"CTXS" ,"LDOS" ,"ASX" ,
                "NICE" ,"FICO" ,"CGNX" ,"PAGS" ,"GDDY" ,"AFRM" ,"BSY" ,"CDAY" ,"OTEX" ,"NLOK" ,"JKHY" ,"BKI" ,"FFIV" ,"TUYA" ,"SEDG" ,"BILL" ,"IPGP" ,"AVLR" ,
                "CREE" ,"OLED" ,"DBX" ,"DOX" ,"MKSI" ,"DLB" ,"PSFE" ,"PFPT" ,"MCFE" ,"PEGA" ,"ESTC" ,"PCTY" ,"RUN" ,"FLEX" ,"ST" ,"KC" ,"JNPR" ,"G" ,"WEX" ,
                "AZPN" ,"DXC" ,"ARW" ,"GLOB" ,"MANH" ,"TIXT" ,"GWRE" ,"CIEN" ,"CNXC" ,"JBL" ,"FSLR" ,"FLIR" ,"PLAN" ,"LPL" ,"LSPD" ,"EEFT" ,"BRKS" ,"COMP" ,"LSCC" ,
                "FOUR" ,"SMAR" ,"CDK" ,"CACI" ,"LAZR" ,"LFUS" ,"IIVI" ,"SNX" ,"COHR" ,"LITE" ,"NCR" ,"BMBL" ,"BL" ,"MSTR" ,"SLAB" ,"CLGX" ,"NTNX" ,"VNT" ,"DAVA" ,
                "OCFT" ,"CRCT" ,"AI" ,"NATI" ,"TDC" ,"JCOM" ,"SWI" ,"NCNO" ,"DSGX" ,"AYX" ,"SAIC" ,"ACVA" ,"DQ" ,"QTWO" ,"CD" ,"PSTG" ,"DCT" ,"POWI" ,"RXT" ,"XRX" ,
                "AMKR" ,"FSLY" ,"ALTR" ,"DOCN" ,"PRSP" ,"ALGM" ,"TTEC" ,"SYNA" ,"CCMP" ,"BB" ,"ASAN" ,"SONO" ,"CYBR" ,"FEYE" ,"NOVT" ,"CRUS" ,"ACIW" ,"AVT" ,"SWCH" ,
                "OLO" ,"ATC" ,"SAIL" ,"API" ,"WK" ,"ALRM" ,"SMTC" ,"MDLA" ,"SABR" ,"MSP" ,"SPWR" ,"FROG" ,"QLYS" ,"TENB" ,"ENV" ,"JAMF" ,"NEWR" ,"DIOD" ,"VICR" ,
                "MTSI" ,"VIAV" ,"ROG" ,"STMP" ,"SPSC" ,"VSH" ,"CLDR" ,"VSAT" ,"COMM" ,"ITRI" ,"MANT" ,"LPSN" ,"NSIT" ,"CRNC" ,"EXLS" ,"VNET" ,"BOX" ,"BLKB" ,"VRNT" ,
                "RAMP" ,"AMBA" ,"FN" ,"CVLT" ,"EGHT" ,"ONTO" ,"NOVA" ,"PD" ,"EVTC" ,"BAND" ,"CRSR" ,"KLIC" ,"FORM" ,"TSEM" ,"ESE" ,"YALA" ,"CALX" ,"IRBT" ,"MIME" ,
                "CSOD" ,"SANM" ,"PLXS" ,"DM" ,"NVMI" ,"MXL" ,"SATS" ,"SVMK" ,"CSIQ" ,"DDD" ,"SIMO" ,"RIOT" ,"PRFT" ,"MFGP" ,"AVYA" ,"SEMR" ,"BTRS" ,"RMBS" ,"HIMX" ,
                "XPER" ,"MVIS" ,"VLDR" ,"PING" ,"UCTT" ,"PRGS" ,"NTCT" ,"KN" ,"EB" ,"MAXR" ,"GB" ,"ZUO" ,"SMCI" ,"ONTF" ,"DSP" ,"PRO" ,"OUST" ,"MEI" ,"OSIS" ,"EPAY" ,
                "SYKE" ,"SUMO" ,"INFN" ,"RAAS" ,"PAR" ,"PLT" ,"TTMI" ,"YEXT" ,"DCBO" ,"CGNT" ,"COHU" ,"SPNS" ,"GPRO" ,"CNDT" ,"UIS" ,"ICHR" ,"VRNS" ,"CSGS" ,"JKS" ,
                "MTLS" ,"EXTR" ,"FORTY" ,"QADA" ,"PLUS" ,"ACLS" ,"MODN" ,"QADB" ,"MLAB" ,"CNXN" ,"RDWR" ,"LINX" ,"FARO" ,"OPRA" ,"AGYS" ,"GSKY" ,"DBD" ,"VCRA" ,"NTGR" ,
                "CEVA" ,"PI" ,"SGH" ,"BHE" ,"SSYS" ,"VECO" ,"IMOS" ,"SCWX" ,"CLS" ,"CTS" ,"AUDC" ,"LASR" ,"IIIV" ,"EBIX" ,"MX" ,"ADTN" ,"PLAB" ,"UEIC" ,"SCSC" ,"INSG" ,
                "MGIC" ,"AOSL" ,"HLIT" ,"ATEN" ,"CASA" ,"PDFS" ,"ABST" ,"ECOM" ,"ZEPP" ,"AMSWA" ,"ALLT" ,"CMTL" ,"BCOV" ,"ITRN" ,"MAXN" ,"GILT" ,"SWIR" ,"DGII" ,
                "VIOT" ,"DMRC" ,"HCKT" ,"SOL" ,"EBON" ,"NPTN" ,"ETWO" ,"CAMP" ,"VPG" ,"VOXX" ,"ZIXI" ,"AXTI" ,"LLNW" ,"DSPG" ,"VIAO" ,"VHC" ,"TUFN" ,"HBB" ,"SILC" ,
                "UEPS" ,"DAKT" ,"CMCM" ,"MIXT" ,"WTRH" ,"CRNT" ,"SOS" ,"KVHI" ,"IMMR" ,"BELFA" ,"BELFB" ,"EXFO" ,"LYTS" ,"CIH" ,"OIIM" ,"SQNS" ,"AVNW" ,"CTG" ,"GSIT" ,
                "ASYS" ,"SREV" ,"SNCR" ,"SPI" ,"CTK" ,"PCTI" ,"RELL" ,"RCAT" ,"TESS" ,"UTSI" ,"MINDP" ,"SEAC" ,"MIND" ,"KNBE" ,"SABRP" ,"IIVIP" ,"APP" ,"VZIO" ,"CTLP" ,
                "STEM" ,"COIN" ,"AVGOP" ,"ALKT" ,"PATH" ,"DV"]
for ticker in tech_tickers:
    # For each ticker, navigate to the page that holds Seeking Alpha's links for transcripts that include that ticker
    page = 'https://seekingalpha.com/search?q=' + ticker + '&type=keyword&tab=transcripts'
    driver.get(page)
    # Pause for a moment, then send the page code off to beautifulsoup
    time.sleep(random.randint(5,20)/10)
    page_source = driver.page_source
    soup = BeautifulSoup(page_source, 'lxml')

    # For each ticker, we want to collect a list of links to the transcipt data for each quarter, the quarter in question,
    # and corresponding date. We initialize those lists here.
    links = []
    quarters = []
    dates = []
    # Collecting all the links from the first transcript page for each ticker:
    for each in soup.find_all("div",{'class':'item-details'}):
        if (operator.contains(each.find('a').text, "Earnings Call Transcript")) & (operator.contains(each.a.text, ticker)):
            links.append("https://seekingalpha.com" + each.find('a').get("href"))
            calldate = each.find('div',{'class':'item-metadata'}).text.replace('\n', "").replace(ticker, "").replace('SA Transcripts', "")
            for day in ['Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday']:
                if (operator.contains(calldate, day)):
                    n = calldate.find(day)
                    calldate = calldate[n:].strip()
            dates.append(calldate)
            if operator.contains(each.find('a').text.lower(), "q1"):
                quarters.append('q1')
            elif operator.contains(each.find('a').text.lower(), "q2"):
                quarters.append('q2')
            elif operator.contains(each.find('a').text.lower(), "q3"):
                quarters.append('q3')
            else:
                quarters.append('q4')
    
    # Each page contains 10 links. We'd like to have 10 transcript links per ticker. 
    # Some transcripts reference other tickers, so not all 10 links on the initial page will be to transcripts of the 
    # ticker in question. In those scenarios we can dig into the next 10 transcript-link pages. 
    # If we can't get 10 links in the first 10 pages, we'll move on to the next ticker. 
    pagenum = 2
    trans_num = len(links) 
    while (trans_num < 10) & (pagenum < 10):
        page = 'https://seekingalpha.com/search?q=' + ticker + '&type=keyword&tab=transcripts' + '#page=' + str(pagenum)
        driver.get(page)
        time.sleep(random.randint(5,20)/10)
        page_source = driver.page_source
        soup = BeautifulSoup(page_source, 'lxml')

        # Collecting all the links from the subsequent transcript pages for each ticker:
        for each in soup.find_all("div",{'class':'item-details'}):
            if (operator.contains(each.find('a').text, "Earnings Call Transcript")) & (operator.contains(each.a.text, ticker)):
                links.append("https://seekingalpha.com" + each.find('a').get("href"))
                calldate = each.find('div',{'class':'item-metadata'}).text.replace('\n', "").replace(ticker, "").replace('SA Transcripts', "")
                for day in ['Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday']:
                    if (operator.contains(calldate, day)):
                        n = calldate.find(day)
                        calldate = calldate[n:].strip()
                dates.append(calldate)
                if operator.contains(each.find('a').text.lower(), "q1"):
                    quarters.append('q1')
                elif operator.contains(each.find('a').text.lower(), "q2"):
                    quarters.append('q2')
                elif operator.contains(each.find('a').text.lower(), "q3"):
                    quarters.append('q3')
                else:
                    quarters.append('q4')
        pagenum += 1 
        trans_num = len(links)
    
    # A pretty hacky solution to get the ordinal dates converted to datetime:
    blank = ''
    dates = [blank.join(calldate.rsplit('th', 1)) for calldate in dates]
    dates = [blank.join(calldate.rsplit('st', 1)) for calldate in dates]
    dates = [blank.join(calldate.rsplit('nd', 1)) for calldate in dates]
    dates = [blank.join(calldate.rsplit('rd', 1)) for calldate in dates]
    dates = [calldate.replace('Augu ', 'August ') for calldate in dates] 
    dates = [calldate.replace('Satuay', 'Saturday') for calldate in dates]
    dates = [calldate.replace('Moay', 'Monday') for calldate in dates]
    dates = [calldate.replace('Suay', 'Sunday') for calldate in dates] 
    dates = [datetime.strptime(calldate, '%A, %B %d %Y') for calldate in dates]
    dates2 = [datetime.strftime(calldate, '%Y-%m-%d') for calldate in dates]
    
    i=0
    # Now that we have the links to (ideally) 10 call transcripts for the ticker,
    # we can follow those lnks and pull down the transcripts.
    for ind, name in enumerate(links):
        page = links[ind]
        driver.get(page)
        time.sleep(0.5)
        page_source = driver.page_source
        trans_soup = BeautifulSoup(page_source, 'html.parser')
        
        # Here's the transcript: 
        trans = trans_soup.find("div",{'data-test-id':'content-container'})
        
        # We'll hard-code the quarter and date information into the filename:
        filename = ticker + "_" + quarters[ind] + "_" +  dates2[ind][0:4] + dates2[ind][5:7] + dates2[ind][8:]

        # Create one text file for each transcript:
        file = open(r"./tech_transcripts/" + filename.lower() + ".txt", 'w')
        file.write(trans.text)
        file.close
        i+=1
    
    print(ticker, i, 'transcripts recieved')

AVNW 10 transcripts recieved
CTG 10 transcripts recieved
GSIT 10 transcripts recieved
ASYS 10 transcripts recieved
SREV 10 transcripts recieved
SNCR 10 transcripts recieved
SPI 0 transcripts recieved
CTK 10 transcripts recieved
PCTI 10 transcripts recieved
RELL 10 transcripts recieved
RCAT 0 transcripts recieved
TESS 10 transcripts recieved
UTSI 10 transcripts recieved
MINDP 0 transcripts recieved
SEAC 10 transcripts recieved
MIND 12 transcripts recieved
KNBE 0 transcripts recieved
SABRP 0 transcripts recieved
IIVIP 0 transcripts recieved
APP 12 transcripts recieved
VZIO 0 transcripts recieved
CTLP 1 transcripts recieved
STEM 6 transcripts recieved
COIN 0 transcripts recieved
AVGOP 0 transcripts recieved
ALKT 0 transcripts recieved
PATH 0 transcripts recieved
DV 10 transcripts recieved
